In [1]:
from IPython.display import clear_output
!pip install pandas tqdm requests httplib2 google-cloud-language
clear_output()
print('All Installed.')


All Installed.


In [2]:
import httplib2
import pandas as pd
import logging
import time
from tqdm import tqdm
import warnings
import datetime as dt
import os
import string
from datetime import timedelta, date
from urllib.error import HTTPError
from IPython.display import clear_output
import requests
import json

from google.cloud import language
from google.oauth2 import service_account
from google.cloud.language import enums
from google.cloud.language import types

client = language.LanguageServiceClient.from_service_account_json('services.json')

In [3]:
def pull_googlenlp(client, url, invalid_types = ['OTHER'], **data):
    
    '''
    Loads HTML from a URL and passes to the Google NLP API
    Parameters:
        * client: Authenticated Language Service Client.
        * url: url of page to load (str)
        * invalid_types: entitity types to ignore (eg. CONSUMER_GOOD, NUMBER) (list) default: OTHER
        * timeout: request timeout in seconds (int) default: 20
    Returns:
    Dictionary of sentiment(list), entities(list), categories(list) (dict)
    '''    

    html = load_text_from_url(url, **data)
    
    if not html:
        return None
    
    document = types.Document(
        content=html,
        type=language.enums.Document.Type.HTML )

    features = {'extract_syntax': True,
                'extract_entities': True,
                'extract_document_sentiment': True,
                'extract_entity_sentiment': True,
                'classify_text': False
                }
    
    response = client.annotate_text(document=document, features=features)
    sentiment = response.document_sentiment
    entities = response.entities
    
    response = client.classify_text(document)
    categories = response.categories
        
    def get_type(type):
        return client.enums.Entity.Type(entity.type).name
    
    result = {}
    
    result['sentiment'] = []    
    result['entities'] = []
    result['categories'] = []

    if sentiment:
        result['sentiment'] = [{ 'magnitude': sentiment.magnitude, 'score':sentiment.score }]
        
    for entity in entities:
        if get_type(entity.type) not in invalid_types:
            result['entities'].append({'name': entity.name, 'type': get_type(entity.type), 'salience': entity.salience, 'wikipedia_url': entity.metadata.get('wikipedia_url', '-')  })
            
    for category in categories:
        result['categories'].append({'name':category.name, 'confidence': category.confidence})
        
        
    return result